In [2]:
import pandas as pd

# --- 0. Initial Dataset (Positive Examples) ---
# Let's assume we have the following (question, coherent_answer) pairs.
# In a real scenario, this would come from SQuAD, Natural Questions, etc.
positive_pairs_data = [
    # --- Product Search ---
    ("Hello, where can I find the milk?", "Hello! Milk is in the dairy section, aisle 4."),
    ("Excuse me, do you have gluten-free pasta?", "Yes, of course. It's in the specialty foods aisle, next to the flours."),
    ("I'm looking for ground coffee, please.", "Ground coffee is at the end of aisle 2, with the teas and powdered chocolates."),
    ("Do you know where the canned tomatoes are?", "Canned tomatoes are in the sauces and condiments aisle, aisle 7."),
    ("Hello, I can't find the spice section.", "Spices are located near the baking aids, in the middle of aisle 6."),
    ("Do you sell AA batteries?", "Yes, batteries are usually near the checkouts or in the electronics/general merchandise section."),
    ("Where is the bakery section located?", "The bakery is at the very back of the store, on your left."),
    ("Do you have whole wheat sliced bread?", "Yes, it's with the other sliced breads in the bakery section."),
    ("I'm looking for extra virgin olive oil.", "Olive oil is in the oils and vinegars aisle, aisle 5."),
    ("Do you have organic plain yogurts?", "Yes, organic yogurts are in the organic section of the dairy aisle."),
    ("Where are the fresh fruits and vegetables?", "That's the first section when you enter the store."),
    ("I'd like some apples, but I only see Golden ones.", "The other apple varieties are right next to them, a bit further down the display."),
    ("Do you have grated cheese?", "Yes, in the cheese counter and pre-packaged cheese section, opposite the yogurts."),
    ("Where can I find eco-friendly dish soap?", "In the cleaning products aisle, look for the 'eco-friendly' or 'organic' label."),
    ("I'm looking for adult cat food.", "The pet food aisle is at the back of the store, aisle 12."),

    # --- Product Information ---
    ("How much is this pack of cookies?", "The price is displayed on the label underneath; it's €2.50."),
    ("Are these apples local?", "Yes, these ones come from a local producer."),
    ("What's the expiration date on this yogurt?", "The date is indicated on the top of the container; here it's the 25th of next month."),
    ("Is this product on sale this week?", "Yes, there's a 20% discount on it; look at the promotional sign."),
    ("Could you tell me if this ready meal contains nuts?", "I'll check the ingredients for you. Yes, it says 'may contain traces of nuts'."),
    ("Is this cheese pasteurized?", "Yes, most of our pre-packaged cheeses are. It's written on the packaging."),
    ("Is this bread made on-site?", "Yes, our bread is kneaded and baked in our bakery every day."),
    ("Do you have a smaller version of this product?", "Let me check. No, sorry, that's the only size we have in stock for this one."),
    ("Is this fish wild-caught or farmed?", "This one is farmed fish. The wild salmon is right next to it."),
    ("Which red wine do you recommend to go with beef bourguignon?", "I recommend this Burgundy Pinot Noir; it will pair very well."),
    ("Is there added sugar in this fruit juice?", "It says 'no added sugar' on the bottle."),
    ("Is this laundry detergent hypoallergenic?", "Yes, it's dermatologically tested and suitable for sensitive skin."),
    ("Is this product vegetarian?", "Yes, it has the V vegetarian label."),
    ("How many grams is this bag of coffee?", "It's a 250-gram bag."),
    ("Are these free-range eggs?", "Yes, the code '1' on the egg indicates they are free-range."),

    # --- Store Services and Operations ---
    ("Hello, is this checkout open?", "Yes, please, you can come forward."),
    ("Do you accept meal vouchers?", "Yes, we accept meal vouchers for food items."),
    ("Do you sell reusable bags?", "Yes, you'll find them near the checkouts."),
    ("Can I pay by contactless card?", "Absolutely, all our checkouts are equipped for it."),
    ("Where are the restrooms, please?", "The restrooms are at the back of the store, next to customer service."),
    ("What are your opening hours on Sunday?", "We are open from 9 AM to 1 PM on Sundays."),
    ("Do you have a loyalty program?", "Yes, you can sign up for free at customer service to receive discounts."),
    ("Is it possible to have groceries delivered?", "Yes, we offer a delivery service. Please inquire at the reception desk."),
    ("I couldn't find a basket, where are they?", "There are some right at the store entrance."),
    ("If an item doesn't have a price, what should I do?", "Bring it to a price checker terminal or ask an employee."),
    ("Can we taste this cheese before buying it?", "At the cheese counter, it's sometimes possible; ask our cheesemonger."),
    ("Is there a pharmacy in the shopping center?", "Yes, there's a pharmacy right outside the supermarket."),
    ("Do you have an ATM?", "Yes, there's one near the main entrance."),
    ("Where can I drop off my glass bottles for recycling?", "We have recycling bins in the parking lot."),
    ("If I'm not satisfied with a product, can I return it?", "Yes, with your receipt, within 15 days for most products."),

    # --- Checkout Interactions ---
    ("Hello, I'll take this, please.", "Hello! Very good."),
    ("Do you have the store card?", "Yes, here it is."),
    ("Would you like a bag?", "Yes, a large one, please. Is there a charge for it?"),
    ("That will be €25.60, please.", "I'll pay by card."),
    ("Would you like the receipt?", "Yes, thank you."),
    ("Do you have change for 50 cents?", "Let me see... Yes, here you go."),
    ("This barcode isn't scanning, do you have another identical item?", "No, that's the last one. Maybe the code is damaged?"),
    ("I forgot my carrots on the belt!", "No problem, I'll add them."),
    ("Do you take holiday vouchers?", "No, sorry, we don't accept them."),
    ("Can I have a separate receipt for these items?", "Of course, tell me which ones."),

    # --- Politeness and Common Phrases ---
    ("Thank you very much!", "You're welcome, have a nice day!"),
    ("Excuse me for bothering you.", "No problem, how can I help you?"),
    ("Have a nice day!", "You too, thank you!"),
    ("Goodbye.", "Goodbye, and see you soon!"),
    ("Excuse me, may I pass?", "Yes, go ahead."),
    ("Are you next?", "No, no, after you, please."),
    ("That's very kind of you.", "My pleasure."),
    ("You're welcome. (in response to 'thank you')", "Thank you."), # Note: "Je vous en prie" can be "You're welcome" or "Please" depending on context. Here it's likely a response.
    ("One moment, please.", "Of course, I'll wait."),
    ("I'll be right there.", "Okay, thank you."),

    # --- Product Availability / Stock ---
    ("Do you still have this product in the back?", "I'll check in the stockroom for you."),
    ("When will you receive this product again?", "Normally, we should have a new delivery tomorrow morning."),
    ("Is there no more almond milk of this brand?", "It seems we're out of stock for now, but we have other brands if you'd like."),
    ("Is this the last pack of this coffee?", "Yes, it's the last one on the shelf currently."),
    ("You don't have Gariguette strawberries anymore?", "No, the season is almost over. We have other varieties."),

    # --- Specific Sections (butcher, fishmonger, cheese counter) ---
    ("Hello, I'd like two slices of cooked ham, please.", "Very good, with or without the rind?"),
    ("Could you prepare 500g of minced beef for me?", "Of course, I'll prepare that for you right away."),
    ("Is this fish fresh today?", "Yes, it arrived this morning."),
    ("I'd like a piece of fruity Comté cheese, about 200g.", "There you go, is this amount okay?"),
    ("Can you gut and scale this sea bass for me?", "Yes, no problem."),
    ("What's the best way to cook this type of fish?", "You can bake it in the oven with herbs and a drizzle of lemon; it's delicious."),
    ("Is this a Label Rouge free-range chicken?", "Yes, it's a free-range chicken."),
    ("Which cheese do you recommend for a raclette?", "Plain raclette cheese is a classic, but we also have smoked or pepper varieties."),
    ("How long can this meat be kept once purchased?", "You should consume it within 2-3 days or freeze it."),
    ("Would it be possible to order a tiered cake for next week?", "Yes, please speak to our pastry chef; he will take your order."),

    # --- Miscellaneous Questions ---
    ("Is parking free?", "Yes, you have 2 hours of free parking."),
    ("Is there Wi-Fi in the store?", "Yes, the network is called 'SupermarcheWifi'; the password is displayed at the reception."),
    ("I've lost my keys, have you found them?", "I'll check at the lost and found. Can you describe them to me?"),
    ("Do you sell gift cards?", "Yes, we have gift cards of different amounts available at the checkout."),
    ("Where is the baby products aisle?", "It's aisle 10, just after the hygiene products."),
    ("Is this product made in France?", "Yes, look for the small blue-white-red flag on the packaging."),
    ("Do you have promotions on Easter chocolates?", "Yes, there's a 'buy 2, get the 3rd free' offer on a selection."),
    ("I'm looking for a specific product, but I don't know the exact name.", "Describe it to me; maybe I can help you find it."),
    ("The music is a bit loud, could you turn it down?", "I'll see what I can do with the manager."),
    ("Do you take back empty gas cylinders?", "Yes, if you buy a full one in exchange."),
    ("Are there charging stations for electric cars in the parking lot?", "Yes, a few spots are equipped near the entrance."),
    ("Can I get an invoice for my business purchases?", "Of course, give me your company information at the checkout."),
    ("Are you closing soon?", "We close in 30 minutes, at 8 PM."),
    ("It's a gift, do you have gift wrap?", "No, sorry, we don't sell it here."),
    ("I don't think this fruit looks very fresh.", "Show it to me; we'll check and replace it if necessary."),
    ("Is there a book or magazine section?", "Yes, a small selection near the checkouts."),
    ("This shopping cart has a stuck wheel.", "Oh, sorry about that. You can grab another one at the entrance."),
    ("Do you have special offers for students?", "Not permanently, but follow our weekly promotions."),
    ("Is the tap water drinkable here?", "Yes, the water is drinkable throughout the city."),
    ("Could you show me where the ice cream aisle is?", "Of course, it's at the back of the store, with the frozen foods, aisle 11."),
    ("I'm looking for flowers, do you sell them?", "Yes, we have a small stand of cut flowers and plants near the entrance."),
    ("This product is out of stock, do you know when it will be back?", "I can check our delivery schedule, one moment. It should arrive on Thursday."),
    ("Are there discounts for large families?", "No, no specific discount, but our loyalty card offers benefits to everyone."),
    ("I'd like to buy alcohol, do I need to show ID?", "Yes, if you appear to be under 25, you might be asked for it."),
    ("Do you have wheelchairs available for customers?", "Yes, ask at the reception desk; we have a few."),
    ("Hello, where can I find the milk, please?", "Hello! Milk is in aisle 3, in the dairy section."),
    ("Excuse me, do you know where the pasta is?", "Yes, pasta is in the grains and starches aisle, number 5, in the middle."),
    ("I'm looking for the cookie aisle.", "The cookie aisle is just past the checkouts, on your left."),
    ("Where are the fruits and vegetables?", "Fruits and vegetables are at the entrance of the store; you can't miss them."),
    ("Could you tell me where the ground coffee is?", "Sure, ground coffee is in aisle 2, with the tea and powdered chocolate."),
    ("Do you know if you have organic tomato sauce?", "Yes, organic tomato sauce is in the same place as other tomato sauces, aisle 5."),
    ("I can't find the whole wheat sliced bread.", "Whole wheat sliced bread is in the bakery section, near the fresh bread."),
    ("Where did you put the eco-friendly cleaning products?", "Eco-friendly cleaning products have their own section at the end of aisle 7."),
    ("Which way to the frozen food section?", "The frozen food section is at the very back of the store, after the dairy products."),
    ("Hello, I'm looking for Greek yogurt.", "Hello, they are in the dairy section, with the other yogurts, aisle 3."),

    # --- Product Availability ---
    ("Do you still have ripe avocados?", "Yes, we have a few left. They are right here."),
    ("Do you have any free-range chicken left?", "Let me check in the back. Yes, we still have some!"),
    ("You wouldn't happen to have any more traditional baguettes?", "Yes, we're just taking them out of the oven now. Help yourself!"),
    ("I'd like some strawberries, but I don't see any more.", "Sorry, we're out of stock of strawberries today."),
    ("Do you have this model of cereal in a large size?", "Normally yes, it should be on the top shelf. Otherwise, I can look in the back."),
    ("Is there no more unsweetened almond milk?", "I'll check. Ah, there's one carton hidden behind! Here it is."),
    ("Are you expecting a delivery of this cheese soon?", "Yes, we should receive some tomorrow morning."),
    ("Do you sell rechargeable AA batteries?", "Yes, they are at the checkouts, on the displays."),
    ("I'm looking for a specific brand of cat food, do you have it?", "Which brand? ... Yes, we have it, it's in the pet aisle."),
    ("This product is on sale, but there's no more. Do you have any in stock?", "I'll see if we can bring some from the stockroom. One moment."),

    # --- Product Information (price, date, etc.) ---
    ("What's the price of these cherry tomatoes, please?", "These cherry tomatoes are €2.50 per punnet."),
    ("Could you tell me the expiration date of this ham?", "Of course, the date is the 15th of next month."),
    ("Are these apples organic?", "Yes, all the apples on this display are organically grown."),
    ("Is this wine dry or rather fruity?", "This one is a rather dry wine. If you're looking for fruity, I recommend the one next to it."),
    ("Is there gluten in this bread?", "No, this one is specifically gluten-free. It's indicated on the packaging."),
    ("How much is a kilo of bananas today?", "Bananas are €1.99 per kilo this week."),
    ("How long is this promotion valid?", "This promotion is valid until Saturday evening inclusive."),
    ("Can I have a thinner slice of this sausage?", "Absolutely, I'll cut that for you right away."),
    ("Is this fish fresh today?", "Yes, it arrived this very morning from the fish market."),
    ("Is the displayed price for the pack of two?", "Yes, that's correct, the price is for the pack."),

    # --- Checkout and Payment ---
    ("Hello, I'll take this.", "Hello! Very well. Do you have the store card?"),
    ("Do you accept meal vouchers?", "Yes, we accept meal vouchers for food items."),
    ("Can I pay by contactless card?", "Yes, of course, up to 50 euros."),
    ("Could I have a bag, please?", "Yes, that will be 10 cents for a reusable bag."),
    ("Can I have the receipt?", "Of course, here it is."),
    ("I think there's an error on the price of this item.", "Let's check that together. Oh yes, excuse me, I'll correct it."),
    ("Do you give change on luncheon vouchers?", "No, sorry, we don't give change on luncheon vouchers."),
    ("Is this checkout open?", "Yes, I'll be with you in a moment."),
    ("I'd like to pay in cash.", "No problem. That will be €12.75."),
    ("Can I use my discount coupons here?", "Yes, present them to me before payment."),

    # --- Services and General Assistance ---
    ("Excuse me, could you help me reach this item up high?", "With pleasure, here you go."),
    ("I dropped a jar of jam, what should I do?", "Don't worry, report it to an employee, we'll clean it up."),
    ("Where are the restrooms, please?", "The restrooms are at the back of the store, near the after-sales service."),
    ("Do you have a home delivery service?", "Yes, for purchases over 100 euros, under certain conditions. Inquire at the reception."),
    ("I'm looking for someone from the butcher department.", "I'll call someone for you. One moment."),
    ("Hello, I have a question about a product I bought yesterday.", "Hello, please go to the reception desk, they can help you."),
    ("Do you have baskets?", "Yes, the baskets are at the entrance, right here."),
    ("Could you tell me where the store manager is?", "The manager is currently in a meeting, but the assistant manager is available at the reception."),
    ("This product is damaged, can I take another one?", "Of course, sorry for the inconvenience. Take another one from the shelf."),
    ("What time do you close tonight?", "We close at 8 PM tonight."),

    # --- Courtesy and Small Interactions ---
    ("Hello!", "Hello, welcome!"),
    ("Thank you very much!", "You're welcome, have a nice day!"),
    ("Excuse me, do you mind?", "Please, go ahead."), # "Vous permettez ?" is like "May I?" or "Do you mind if I pass?"
    ("After you.", "Thank you, that's kind."),
    ("It's nice weather today, isn't it?", "Yes, finally a bit of sun!"),
    ("Have a nice day!", "You too!"),
    ("Did you have a good day?", "Yes, thank you, and you?"),
    ("Excuse me for bothering you.", "No worries, what can I do for you?"),
    ("I wish you good luck/strength. (contextual, often for work)", "Thank you, you too."), # "Bon courage" is hard to translate directly, means "keep up the good work / hang in there"
    ("Goodbye!", "Goodbye, and see you soon!"),

    # --- More specific product searches ---
    ("Hello, I can't find the gherkins.", "Hello, gherkins are with the condiments, aisle 4, next to the mustard."),
    ("Where can I find plastic wrap?", "Plastic wrap is in the household products aisle, with the aluminum foil, aisle 7."),
    ("Do you have red onions?", "Yes, red onions are with the other onions and garlic, in the vegetable section."),
    ("I'm looking for basmati rice.", "Basmati rice is in the world foods aisle or with the other rices, aisle 5."),
    ("Where are the dishwasher tablets?", "Dishwasher tablets are in the cleaning products aisle, number 7."),
    ("I'd like some grated cheese.", "Grated cheese is in the dairy section, with the other pre-packaged cheeses, aisle 3."),
    ("Do you have E27 LED bulbs?", "Yes, bulbs are usually near the checkouts or in a small hardware section if we have one."),
    ("I don't see the granulated sugar.", "Granulated sugar is in the sweet grocery aisle, with the flour, aisle 2."),
    ("Where did you put the pitted green olives?", "Olives are with the snacks or condiments, aisle 4."),
    ("I'm looking for extra virgin olive oil.", "Olive oil is in the oils and vinegars aisle, also aisle 4."),

    # --- More questions about availability ---
    ("Do you have any of this special salt-free bread left?", "I'm afraid not for today, it sells out very quickly. Try tomorrow morning."),
    ("You don't have any more Coca-Cola Zero in cans?", "Yes, look carefully on the bottom shelf, there should be some left."),
    ("Is this the last pack of water?", "No, we still have stock in the back, I can bring you some if needed."),
    ("Have you received the new weekly promotions?", "Yes, they are indicated by special labels on the shelves."),
    ("I can't find the laundry detergent I usually buy, have you discontinued it?", "It's possible it's temporarily out of stock or the packaging has changed. What brand?"),

    # --- More questions about products ---
    ("Is this melon perfectly ripe?", "This one seems good to me; it's slightly soft at the end."),
    ("What's the difference between these two types of smoked salmon?", "This one is from Scotland, milder; the other from Norway, a bit smokier."),
    ("Are these free-range eggs?", "Yes, the code '1' on the egg indicates it, and it's marked on the box."),
    ("Does this ready meal contain nuts?", "You need to check the allergen list on the label. Here, let's look together."),
    ("Does the 'buy 2, get the 3rd free' promotion apply to this item?", "Yes, that's indeed the case for this product."),

    # --- More checkout interactions ---
    ("Hello, I'd also like a pack of chewing gum.", "Of course, which one would you like? They're over there."),
    ("Do you take American Express?", "Sorry, we don't take American Express here."),
    ("I forgot my wallet, can you put my groceries aside?", "Yes, we can keep them at the reception for an hour or two."),
    ("The payment terminal isn't working.", "Let's try another one. There, that should work."),
    ("I'd like to make a donation to the charity at the checkout.", "With pleasure, how much would you like to donate?"),

    # --- Customer-customer interactions (simple) ---
    ("Excuse me, are you in line?", "Yes, this is it."),
    ("Excuse me, could you pass me that product up high?", "Sure, here you go."),
    ("Do you know if this product is good?", "I buy it regularly, it's very good!"),
    ("Watch out, you dropped something.", "Oh, thank you so much for letting me know!"),
    ("Is this the line for the express checkout?", "No, the express checkout is the one next to it."),
    # --- More specific interactions at service counters ---
    # Butcher / Deli
    ("Hello, I'd like 500 grams of minced beef, please.", "Hello! 5% or 15% fat?"),
    ("Could you cut me four rather thin slices of Parma ham?", "Of course. Like this, is the thickness okay?"),
    ("What cut do you recommend for a good pot-au-feu (beef stew)?", "For a pot-au-feu, I recommend shank or chuck, it's excellent."),
    ("Is this pâté homemade?", "Yes, this one is prepared by our teams in our kitchen."),
    ("I'd like a rotisserie chicken, please. Which one is the hottest?", "This one just came out of the oven, it's nice and hot!"),
    ("Do you have herb chipolatas today?", "Yes, they're right here. How many would you like?"),
    ("I'll take about 200g of duck rillettes.", "Here are the rillettes. Anything else with this?"),

    # Fishmonger
    ("Hello, are these cod fillets indeed boneless?", "Hello, yes, these are loins, normally boneless, but always double-check."),
    ("Could you prepare two sea breams for me? Gutted and scaled.", "Absolutely, I'll do that for you right away."),
    ("What's the fish of the day on special?", "Today, we have hake on special at €12.90 per kilo."),
    ("Are these shrimp cooked or raw?", "These ones are already cooked. The raw ones are right next to them."),
    ("I'm looking for oysters, do you have any special ones?", "Yes, we have Fines de Claire and Gillardeau. Which ones do you prefer?"),

    # Cheese Counter
    ("Hello, I'd like a piece of fruity Comté, about 300 grams.", "Hello. I have this one, aged 18 months, it's excellent. Would that suit you?"),
    ("What cheese do you recommend for a raclette for 6 people?", "For 6, count about 200-250g per person. I can make you an assortment."),
    ("Is this goat cheese very dry or rather creamy?", "That one is a fresh goat cheese, so quite creamy. If you want drier, I have this one."),
    ("Can I taste this Roquefort before buying it?", "Of course, here, have a small piece."),
    ("Do you have 'fromage blanc à la faisselle' (curdled fresh cheese)?", "Yes, it's right there. In 500g or 1kg pots."),

    # Bakery / Patisserie (if specific stand)
    ("Hello, do you have any butter croissants left?", "Hello, yes, they're just coming out of the oven! How many would you like?"),
    ("I'll take a traditional baguette, not too well-done, please.", "Here you go, it's still warm."),
    ("Do you make birthday cakes to order?", "Yes, you need to order 48 hours in advance. You can see our catalog here."),
    ("This strawberry tart, how many people is it for?", "This one is for 6 to 8 people."),
    ("Do you have sliced whole wheat bread?", "Yes, I can slice this one for you if you wish."),

    # --- Questions about promotions and loyalty card ---
    ("Hello, I saw this product was 'buy 2, get the 3rd free'. How does that work?", "Hello, you take three identical items, and the cheapest of the three is free at the checkout."),
    ("Does the 10% discount apply to all organic products today?", "Yes, on the entire organic section, excluding promotions already underway."),
    ("How can I know if my loyalty card has been scanned correctly?", "It will be indicated on your receipt, and your points will be credited."),
    ("I have a 5 euro discount voucher, can it be combined with current promotions?", "Normally yes, unless otherwise stated on the voucher. Present it at the checkout."),
    ("Where can I find the weekly promotional catalog?", "It's available at the store entrance or on our website."),

    # --- Minor problems and requests for assistance ---
    ("Excuse me, I dropped a jar. What should I do?", "Don't worry, don't touch anything, I'll call someone to clean it up."),
    ("This pack of cookies is open, can I take another one?", "Of course, sorry about that. Help yourself, I'll remove this one."),
    ("The scale for weighing vegetables seems not to be working.", "Ah, I'll check. In the meantime, the one in the next aisle is working."),
    ("I can't scan this item at the self-checkout.", "I'm coming to help you. Sometimes the barcode is a bit damaged."),
    ("There are no more bags for fruits and vegetables here.", "I'll bring you some right away."),

    # --- Questions about origin, composition, specific diets ---
    ("Where do these tomatoes come from, please?", "These tomatoes come from Spain. The French ones are right next to them."),
    ("Does this ready meal contain palm oil?", "You need to check the ingredient list. Here, we can check together."),
    ("Do you have a clearly identified range of lactose-free products?", "Yes, we have a specific 'lactose-free' section in aisle 2, near the organic section."),
    ("I'm looking for vegan cookies, where can I find them?", "In the organic section, we have a selection of vegan cookies."),
    ("Is this chicken Label Rouge certified?", "Yes, it's indicated on the label, it has the Label Rouge certification."),

    # --- Slightly more specific or less common requests ---
    ("Hello, is it possible to order a product that you don't have in stock?", "It depends on the product. Inquire at the reception; they can tell you."),
    ("Do you have a locker service for backpacks?", "No, sorry, we don't have a locker service here."),
    ("I'm looking for cut flowers, where are they?", "Flowers are usually near the entrance or the checkouts."),
    ("Do you sell store gift cards?", "Yes, you can buy them at the reception, for the amount of your choice."),
    ("Do you take back returnable glass bottles?", "Yes, the bottle return machine is near the entrance, next to the carts."),

    # --- End of transaction and additional courtesies ---
    ("Thank you, that was perfect. Do you need help bagging?", "No thank you, I'll be fine. Have a nice day!"),
    ("I don't need a receipt, thank you.", "Very well, have a good rest of your day!"),
    ("Can I have an invoice in my company's name?", "Yes, give me the information at the reception after you've paid at the checkout."),
    ("Are you sure there's no mistake? It seems quite cheap to me.", "I've scanned everything correctly. Enjoy the promotions!"),
    ("Thank you for your patience, it was busy!", "You're welcome, it's normal. See you soon!"),

    # --- Customer-customer interactions (a bit more) ---
    ("Excuse me, is this the line for the '10 items or less' checkout?", "Yes, absolutely, you're in the right place."),
    ("Excuse me, would you know if this product is on sale?", "I think I saw it on an endcap display with a red tag."),
    ("Do you mind if I go ahead of you? I only have one item.", "Yes, go ahead, no problem."),
    ("Oh, you're taking the last one! I wanted one too.", "Oh, sorry! Maybe there are more in the stockroom?"),
    ("Have you seen where they put the new organic coffee?", "I think it's at the end of the usual coffee aisle now."),
]

# Let's check the size
print(f"Number of generated pairs: {len(positive_pairs_data)}")

Number of generated pairs: 247


In [3]:
negative_pairs_data = [
    # --- Product Search vs Random Answers ---
    ("Hello, where can I find the milk?", "The next train to Paris will depart at 2:30 PM."),
    ("Excuse me, do you have gluten-free pasta?", "I've always dreamed of visiting the moon."),
    ("I'm looking for ground coffee, please.", "Dinosaurs disappeared 65 million years ago."),
    ("Do you know where the canned tomatoes are?", "My favorite color is turquoise blue."),
    ("Hello, I can't find the spice section.", "Don't forget to turn off the light when you leave."),
    ("Do you sell AA batteries?", "The elections will be held next month."),
    ("Where is the bakery section located?", "I love listening to jazz on Sunday mornings."),
    ("Do you have whole wheat sliced bread?", "The Pythagorean theorem is fundamental in geometry."),
    ("I'm looking for extra virgin olive oil.", "My cat's name is Caramel and he's very playful."),
    ("Do you have organic plain yogurts?", "We should think about repainting the living room soon."),

    # --- Product Information vs Off-topic Answers ---
    ("How much is this pack of cookies?", "Mount Everest is the highest peak in the world."),
    ("Are these apples local?", "Remember to take your umbrella, it might rain."),
    ("What's the expiration date on this yogurt?", "I think aliens exist."),
    ("Is this product on sale this week?", "The rules of the road must be respected by everyone."),
    ("Could you tell me if this ready meal contains nuts?", "The capital of Australia is Canberra."),
    ("Is this cheese pasteurized?", "I have a dentist appointment tomorrow."),
    ("Is this bread made on-site?", "Football is a very popular sport."),
    ("Do you have a smaller version of this product?", "Bees are essential for pollination."),
    ("Is this fish wild-caught or farmed?", "Stoic philosophy advocates self-control."),
    ("Which red wine do you recommend to go with beef bourguignon?", "I forgot my keys at home this morning."),

    # --- Store Services and Operations vs Absurd Answers ---
    ("Hello, is this checkout open?", "Penguins can't fly, but they swim very well."),
    ("Do you accept meal vouchers?", "My horoscope says I'm going to meet someone interesting."),
    ("Do you sell reusable bags?", "The golden ratio is approximately 1.618."),
    ("Can I pay by contactless card?", "Ducks go quack-quack."),
    ("Where are the restrooms, please?", "The French Revolution began in 1789."),
    ("What are your opening hours on Sunday?", "I wonder if fish get thirsty."),
    ("Do you have a loyalty program?", "The alphabet contains 26 letters."),
    ("Is it possible to have groceries delivered?", "The sun rises in the east."),
    ("I couldn't find a basket, where are they?", "A triangle has three sides."),
    ("If an item doesn't have a price, what should I do?", "My favorite dish is four-cheese pizza."),

    # --- Checkout Interactions vs Unrelated Answers ---
    ("Hello, I'll take this, please.", "Have you seen the latest superhero movie?"),
    ("Do you have the store card?", "It's beautiful weather for a walk in the forest."),
    ("Would you like a bag?", "I think I left the gas on."),
    ("That will be €25.60, please.", "Do shooting stars really make wishes come true?"),
    ("Would you like the receipt?", "I urgently need a vacation."),

    # --- Politeness vs Strange Answers ---
    ("Thank you very much!", "Carrots are good for your eyesight, apparently."),
    ("Excuse me for bothering you.", "A circle is a perfectly round figure."),
    ("Have a nice day!", "Do you know how to plant cabbages?"), # Literal translation of a children's song
    ("Goodbye.", "The atomic number of gold is 79."),
    ("Excuse me, may I pass?", "I have a very rare stamp collection."),

    # --- Product Availability vs Philosophical/Existential Answers ---
    ("Do you still have this product in the back?", "What is the meaning of life?"),
    ("When will you receive this product again?", "Time is an illusion, isn't it?"),
    ("Is there no more almond milk of this brand?", "Is the universe infinite?"),
    ("Is this the last pack of this coffee?", "If a tree falls in the forest and no one is around to hear it..."),
    ("You don't have Gariguette strawberries anymore?", "Is destiny pre-written?"),

    # --- Specific Sections vs Unrelated Answers ---
    ("Hello, I'd like two slices of cooked ham.", "The solar system has eight planets."),
    ("Could you prepare 500g of minced beef for me?", "I need to remember to buy new socks."),
    ("Is this fish fresh today?", "Poetry is the art of evoking and suggesting sensations."),
    ("I'd like a piece of fruity Comté cheese, about 200g.", "The traffic jams this morning were terrible."),
    ("Can you gut and scale this sea bass for me?", "I'm starving!"),

    # --- Miscellaneous Questions vs Whimsical Answers ---
    ("Is parking free?", "Do unicorns exist in a parallel dimension?"),
    ("Is there Wi-Fi in the store?", "My dog ate my homework yesterday."),
    ("I've lost my keys, have you found them?", "Will teleportation be possible one day?"),
    ("Do you sell gift cards?", "I speak fluent Klingon."),
    ("Where is the baby products aisle?", "The Big Bang occurred about 13.8 billion years ago."),

    ("Where can I find children's cereals?", "The next flight to the moon is in preparation."),
    ("Do you have ice cubes, please?", "I've always wanted to learn to play the harp."),
    ("I'm looking for paper towels.", "Jellyfish don't have brains."),
    ("Do you know where the birthday candles are?", "My favorite dish is sauerkraut with sausages."),
    ("Hello, do you have green tea with mint?", "Don't forget to vote in the next municipal elections."),
    ("I can't find the chips aisle.", "Koalas sleep about 20 hours a day."),
    ("Where are the baby food jars?", "I have a soft spot for 80s science fiction movies."),
    ("Do you have pitted olives?", "The water cycle is a fascinating natural process."),
    ("I'm looking for plastic wrap.", "My dog is afraid of the vacuum cleaner."),
    ("Do you sell trash bags?", "Remember to change the clocks this weekend."),

    # --- Product Information vs Whimsical Answers ---
    ("Is this melon ripe?", "The speed of light is about 300,000 km/s."),
    ("Is this bread baked today?", "If I were an animal, I'd be a golden eagle."),
    ("What's the difference between these two detergents?", "I read a fascinating book about the history of samurai."),
    ("Does this product contain palm oil?", "Last night's concert was absolutely incredible."),
    ("Could you tell me the price per kilo of potatoes?", "The Mona Lisa is exhibited at the Louvre Museum."),
    ("This cheese has a strong smell, is that normal?", "My neighbor makes a lot of noise at night."),
    ("Is this apple juice 100% pure juice?", "I'm planning to repaint my kitchen yellow."),
    ("Do you have a salt-free option for this product?", "The Northern Lights are a magnificent sight."),
    ("Is this fish suitable for sushi?", "The next James Bond movie will be released next year."),
    ("The 70% dark chocolate, is this it?", "I lost my glasses somewhere in the house."),

    # --- Store Services and Operations vs Absurd and Existential Answers ---
    ("Is there a priority checkout for pregnant women?", "Do socks really disappear in the washing machine?"),
    ("Do you accept payment by check?", "If a word is misspelled in the dictionary, how do we know?"),
    ("Do you have smaller shopping carts?", "Why don't sheep shrink when it rains?"),
    ("Can I leave my cart here for a moment?", "Does the universe have a smell?"),
    ("Where is the store manager?", "If time is money, can you withdraw time from an ATM?"),
    ("What are the butcher's opening hours?", "Do birds dream in color?"),
    ("Do you have a locker service?", "If I eat myself, do I double in size?"),
    ("Is it possible to order a birthday cake?", "Why is the sky blue and not green?"),
    ("I'm looking for the battery recycling bins.", "Do clouds have weight?"),
    ("If a product is expired, what do you do with it?", "Do fish drink water?"),

    # --- Checkout Interactions vs Poetic or Childish Answers ---
    ("Hello, just these two items.", "The wind whispers secrets to the leaves of the trees."),
    ("I have my own loyalty card.", "Butterfly, fly, fly, little blue butterfly."),
    ("No need for a bag, thank you.", "Stars are glitter in the dark night."),
    ("How much is that in total?", "One, two, three, to the woods we go."),
    ("I'll pay in cash.", "Once upon a time, there was a princess in a castle."),

    # --- Politeness vs Technical or Scientific Answers ---
    ("Thank you kindly!", "DNA is a helical macromolecule."),
    ("Pardon me.", "The chemical formula for water is H2O."),
    ("Good afternoon!", "Photosynthesis converts light energy into chemical energy."),
    ("With pleasure.", "The quicksort algorithm has an average complexity of O(n log n)."),
    ("Excuse me, are you an employee?", "Newton's third law states that for every action, there is an equal and opposite reaction."),

    # --- Product Availability vs Answers on Random Topics ---
    ("You don't have any more basmati rice?", "The last football match was very close."),
    ("When do you receive French strawberries?", "I learned a new lasagna recipe."),
    ("Is there no more coconut milk?", "Animal documentaries are my favorites."),
    ("Is this the last pack of sparkling water?", "Fashion this summer will be all about bright colors."),
    ("Do you still have promotions on ice cream?", "I really should exercise more often."),

    # --- Specific Sections vs Historical or Geographical Answers ---
    ("Hello, I'd like a rotisserie chicken.", "The Great Wall of China being visible from space is a myth."),
    ("Could you cut me 300g of Gruyère cheese?", "Napoleon Bonaparte died on Saint Helena."),
    ("Is this salmon farmed?", "The Nile is the longest river in Africa."),
    ("I'd like a baguette that's not too well-done.", "The city of Rome was founded, according to legend, in 753 BC."),
    ("Can you recommend a wine to go with fish?", "The Sahara is the largest hot desert in the world."),

    # --- Miscellaneous Questions vs Answers about Fictional Characters ---
    ("Is the store open on May 1st?", "Sherlock Holmes resided at 221B Baker Street."),
    ("Are there promotions for Mother's Day?", "Darth Vader is Luke Skywalker's father."),
    ("I found this wallet on the floor.", "The Cheshire Cat can disappear and reappear at will."),
    ("Do you sell stamps?", "Wonder Woman comes from the island of Themyscira."),
    ("Where is the school supplies aisle?", "Gandalf is a powerful wizard in The Lord of the Rings."),
    ("Hello, where are the light bulbs?", "I've been collecting butterflies since I was a child."),
    ("Excuse me, do you have dental floss?", "I love doing pottery on weekends."),
    ("I'm looking for matches, please.", "Knitting is a very relaxing activity."),
    ("Do you know where the air fresheners are?", "My marathon record is 4 hours and 15 minutes."),
    ("Hello, I can't find the bandages aisle.", "I'm learning to play the ukulele at the moment."),
    ("Do you sell coffee filters?", "Fishing relaxes me enormously."),
    ("Where is the party supplies aisle located?", "I'm passionate about astrophotography."),
    ("Do you have paper tissues?", "I'm part of a very active book club."),
    ("I'm looking for baking soda.", "I really enjoy hiking in the mountains."),
    ("Do you have sponges?", "I'm taking salsa dance lessons."),

    # --- Product Information vs Answers about Dreams/Aspirations ---
    ("How much is this bottle of water?", "I dream of traveling around the world in a hot air balloon."),
    ("Are these oranges sweet?", "One day, I'll write a bestselling novel."),
    ("What's the expiration date on this milk?", "I'd like to learn to fly a plane."),
    ("Is this product on sale this week?", "I'd like to live in a lighthouse."),
    ("Could you tell me if this sunscreen protects against UVA and UVB rays?", "I hope to see dolphins in the wild someday."),
    ("Is this bread salt-free?", "I dream of building my own treehouse."),
    ("Is this detergent biodegradable?", "I'd love to be fluent in several languages."),
    ("Do you have an organic version of this product?", "My biggest wish is to see an aurora borealis."),
    ("Does this fish come from sustainable fishing?", "I'd like to create an association to help animals."),
    ("The coffee beans you have, what's their origin?", "I aspire to a simpler life, close to nature."),

    # --- Store Services and Operations vs Answers about Weather/Seasons ---
    ("Hello, do you do home deliveries?", "Autumn is my favorite season, with its warm colors."),
    ("Do you accept vouchers from this magazine?", "Apparently, it's going to snow next week."),
    ("Do you have a dry-cleaning service?", "I love long summer evenings."),
    ("Can I get an invoice for my purchases?", "The wind is blowing hard today, isn't it?"),
    ("Where are the fitting rooms?", "Spring is synonymous with renewal."),
    ("What are your exceptional closing days?", "I don't much like it when it rains for days."),
    ("Do you have a parcel pick-up point?", "In winter, there's nothing like a good fireplace."),
    ("Is it possible to order online and pick up in store?", "The sun is beating down, remember to stay hydrated."),
    ("I'm looking for the after-sales service.", "The first morning frosts have arrived."),
    ("If an item is defective, can I exchange it?", "I'm looking forward to summer barbecues."),

    # --- Checkout Interactions vs Random Object Answers ---
    ("Hello, I only have this.", "A chair, a table, a lamp."),
    ("Do you take American Express?", "Key, wallet, phone."),
    ("I'd like a plastic bag, please.", "Book, pen, notebook."),
    ("That's €18.50.", "Fork, knife, plate."),
    ("Keep the change.", "Cloud, star, moon."),

    # --- Politeness vs Place Name Answers ---
    ("Thank you infinitely!", "Paris, London, New York."),
    ("Excuse me for the disturbance.", "Mountain, forest, ocean."),
    ("Have a good rest of your day!", "Kitchen, living room, bedroom."),
    ("Goodbye and thank you.", "School, office, park."),
    ("You're welcome.", "Italy, Spain, Japan."),

    # --- Product Availability vs Answers about Emotions/Feelings ---
    ("Do you still have baker's yeast?", "I'm feeling cheerful today."),
    ("When will you receive the new promotions?", "Sometimes, nostalgia gets to me."),
    ("Is there no more of your brand of orange juice?", "The excitement before a trip is incomparable."),
    ("Is this the last jar of strawberry jam?", "The serenity of a sunrise is soothing."),
    ("You don't have any more toilet paper on sale?", "Frustration can be a driver of change."),

    # --- Specific Sections vs Answers about Shapes/Colors ---
    ("Hello, I'd like two minced beef patties.", "A red circle, a blue square, a green triangle."),
    ("Could you recommend a mild cheese?", "Black and white stripes, multicolored polka dots."),
    ("Is this cod very fresh?", "An orange spiral, a yellow star."),
    ("I'd like a traditional baguette, please.", "A gradient of purple, a touch of pink."),
    ("Can you slice this sausage thinly for me?", "Geometric shapes and primary colors."),

    # --- Miscellaneous Questions vs Answers with Names of Historical/Famous People ---
    ("What time does the store close tonight?", "Cleopatra, Julius Caesar, Napoleon."),
    ("Are there disabled toilets?", "Marie Curie, Albert Einstein, Leonardo da Vinci."),
    ("I've lost my child, can you make an announcement?", "Mozart, Beethoven, Bach."),
    ("Do you sell transport cards?", "Simone Veil, Nelson Mandela, Mother Teresa."),
    ("Where is the nearest emergency exit?", "Charlie Chaplin, Marilyn Monroe, Elvis Presley.")
]

# Let's check the size
print(f"Number of generated pairs: {len(negative_pairs_data)}")

Number of generated pairs: 165


In [4]:
negative_pairs_data.extend([
    # --- Product Search (keywords repeated absurdly) ---
    ("Hello, where can I find the milk?", "Milk is a white liquid. Are you looking for milk to drink milk?"),
    ("Excuse me, do you have gluten-free pasta?", "Gluten-free pasta is pasta that does not contain gluten. This pasta is specific."),
    ("I'm looking for ground coffee, please.", "Ground coffee is coffee that has been ground. Coffee is ground to obtain ground coffee."),
    ("Do you know where the canned tomatoes are?", "Canned tomatoes contain tomatoes in a can. These tomatoes are in cans."),
    ("Hello, I can't find the spice section.", "Spices are aromatic substances. The spice section is where spices are sold."),
    ("Do you sell AA batteries?", "AA batteries are a type of battery. We sell batteries, but are they AA batteries?"),
    ("Where is the bakery section located?", "The bakery is where bread is made. This section is dedicated to the bakery."),
    ("Do you have whole wheat sliced bread?", "Whole wheat sliced bread is sliced bread made with whole wheat flour. It's a type of bread."),
    ("I'm looking for extra virgin olive oil.", "Extra virgin olive oil is a quality of olive oil. This oil is extra virgin."),
    ("Do you have organic plain yogurts?", "Organic plain yogurts are plain and organic yogurts. These yogurts are therefore organic."),

    # --- Product Information (tautological answers or definitions) ---
    ("How much is this pack of cookies?", "The cost of this pack of cookies is its price. These cookies have a cost."),
    ("Are these apples local?", "If these apples are local, it means they come from the locality. Local apples are grown locally."),
    ("What's the expiration date on this yogurt?", "The expiration date of this yogurt is the date until which this yogurt is good to consume before expiring."),
    ("Is this product on sale this week?", "If this product is on sale, then there is a sale on this product this week."),
    ("Could you tell me if this ready meal contains nuts?", "If this ready meal contains nuts, then there are nuts in the preparation of this meal."),
    ("Is this cheese pasteurized?", "A pasteurized cheese is a cheese that has undergone pasteurization. Is this cheese pasteurized?"),
    ("Is this bread made on-site?", "Made on-site means that this bread was made here, on site. Is the bread made on-site?"),
    ("Do you have a smaller version of this product?", "A smaller version of this product would be this product, but smaller. We do not have a smaller version of this product if this product does not exist in a smaller version."),
    ("Is this fish wild-caught or farmed?", "A wild fish is not farmed, and a farmed fish is not wild. This fish is either one or the other."),
    ("Which red wine do you recommend to go with beef bourguignon?", "A red wine for beef bourguignon? Beef bourguignon is a dish that can be accompanied by red wine."),

    # --- Store Services and Operations (answers that repeat the question) ---
    ("Hello, is this checkout open?", "Are you asking if this checkout is open? An open checkout is a checkout that is working."),
    ("Do you accept meal vouchers?", "Meal vouchers? You want to know if we accept meal vouchers as a payment method for your restaurant items?"), # "votre restaurant" is a bit odd in the original, kept the slightly odd "your restaurant items"
    ("Do you sell reusable bags?", "Reusable bags for sale? Yes, a reusable bag is a bag that can be reused."),
    ("Can I pay by contactless card?", "Pay by contactless card? It's a card payment that doesn't require contact."),
    ("Where are the restrooms, please?", "The restrooms? Are you looking for the location of the restrooms in this place? Restrooms are sanitary facilities."),

    # --- Checkout Interactions (redundant answers) ---
    ("Hello, I'll take this, please.", "You'll take this? Okay, so you're taking this item."),
    ("Do you have the store card?", "The store card? It's a card for this store. Do you have the store's loyalty card?"),
    ("Would you like a bag?", "A bag? Are you asking me if I want a bag, or are you asking if you want a bag?"),
    ("That will be €25.60, please.", "The total is €25.60. This amount of €25.60 is what you have to pay."),
    ("Would you like the receipt?", "The receipt? It's the proof of your purchase at the checkout. I'll give you the receipt."),

    # --- Product Availability / Stock (answers that rephrase without informing) ---
    ("Do you still have this product in the back?", "This product in the back? If it's in the back, it means we have this product, but in the back."),
    ("When will you receive this product again?", "Receiving this product again means a new delivery of this product. This product will be received later."),
    ("Is there no more almond milk of this brand?", "No more almond milk of this brand? So, the almond milk of this specific brand is no longer available."),
    ("Is this the last pack of this coffee?", "The last pack of this coffee? If it's the last one, then there are no more packs of this coffee."),
    ("You don't have Gariguette strawberries anymore?", "No more Gariguette strawberries? So Gariguette strawberries are no longer in stock currently."),

    # --- Specific Sections (butcher, fishmonger, cheese counter) ---
    ("Hello, I'd like two slices of cooked ham.", "Two slices of cooked ham? Cooked ham can be cut into slices. You want two of them."),
    ("Could you prepare 500g of minced beef for me?", "500g of minced beef? That's a quantity of beef that has been minced. So, prepare 500g."),
    ("Is this fish fresh today?", "Fresh today, this fish? A fish fresh today is a fish caught or arrived today."),
    ("I'd like a piece of fruity Comté cheese, about 200g.", "A 200g piece of fruity Comté? Fruity Comté is a type of Comté. You want a piece."),
    ("Can you gut and scale this sea bass for me?", "Gut and scale this sea bass? That means removing the scales and entrails from this sea bass fish."),
    ("Where can I find children's cereals?", "Children's cereals are cereals intended for children. Looking for these cereals implies finding them."),
    ("Do you have ice cubes, please?", "Ice cubes are frozen water. Having ice cubes means possessing these pieces of ice."),
    ("I'm looking for paper towels.", "Paper towels are paper that absorbs. So you are looking for this paper that has the ability to absorb."),
    ("Do you know where the birthday candles are?", "Birthday candles are candles used for birthdays. The location of these candles is what you're looking for."),
    ("Hello, do you have green tea with mint?", "Green tea with mint is green tea flavored with mint. Having this tea means we offer it."),
    ("I can't find the chips aisle.", "The chips aisle is the section where chips are sold. Not finding it means you don't see this aisle."),
    ("Where are the baby food jars?", "Baby food jars are compotes made for babies. The place where they are located is your question."),
    ("Do you have pitted olives?", "Pitted olives are olives from which the pit has been removed. Having these olives means they are available."),
    ("I'm looking for plastic wrap.", "Plastic wrap is a food film that stretches. Looking for this film implies wanting to locate it."),
    ("Do you sell trash bags?", "Trash bags are bags intended for trash cans. Selling these bags is our business if we have them."),

    # --- Product Information (repetitions and questions in return) ---
    ("Is this melon ripe?", "A ripe melon is a melon that has reached maturity. Does this melon seem ripe to you?"),
    ("Is this bread baked today?", "If this bread is baked today, its baking took place today. Bread baked today is therefore fresh."),
    ("What's the difference between these two detergents?", "The difference between these two detergents lies in what differentiates them. These two detergents are not identical."),
    ("Does this product contain palm oil?", "Palm oil in this product? If this product contains it, then palm oil is one of its ingredients."),
    ("Could you tell me the price per kilo of potatoes?", "The price per kilo of potatoes is the cost for one kilogram of these potatoes. This price is displayed for the potatoes."),
    ("This cheese has a strong smell, is that normal?", "A strong smell for this cheese? Some cheeses have a strong smell. Is the smell of this cheese normal for this type of cheese?"),
    ("Is this apple juice 100% pure juice?", "A 100% pure apple juice contains only apple juice. Is this the case for this apple juice?"),
    ("Do you have a salt-free option for this product?", "A salt-free option for this product would be this product, but without added salt. Having a salt-free option means it exists."),
    ("Is this fish suitable for sushi?", "A fish suitable for sushi is a fish that can be used to make sushi. Is this fish sushi-grade?"),
    ("The 70% dark chocolate, is this it?", "70% dark chocolate contains 70% cocoa. If this is it, then this dark chocolate is 70%."),

    # --- Store Services and Operations (vague reformulations) ---
    ("Is there a priority checkout for pregnant women?", "A priority checkout for pregnant women is a checkout that gives priority to pregnant women. The existence of this checkout is your query."),
    ("Do you accept payment by check?", "Payments by check are a payment method. Accepting these payments means we take them."),
    ("Do you have smaller shopping carts?", "Smaller shopping carts would be carts of a reduced size. Having these carts means they are available."),
    ("Can I leave my cart here for a moment?", "Leaving your cart here for a moment means temporarily abandoning your cart in this spot. Is it possible to leave the cart?"),
    ("Where is the store manager?", "The store manager is the person in charge of the store. Finding this manager is your goal."),

    # --- Product Availability (useless pseudo-confirmations) ---
    ("You don't have any more basmati rice?", "No more basmati rice means an out-of-stock situation for basmati rice. So, basmati rice is no longer there."),
    ("When do you receive French strawberries?", "Receiving French strawberries implies a delivery of these strawberries. French strawberries will arrive when they are received."),
    ("Is there no more coconut milk?", "If there is no more coconut milk, it's because coconut milk is absent. So coconut milk is missing."),
    ("Is this the last pack of sparkling water?", "The last pack of sparkling water? If it's the last one, then there's no other pack of sparkling water after this one."),
    ("Do you still have promotions on ice cream?", "Promotions on ice cream? If we still have promotions on ice cream, then ice cream is still on sale."),

    # --- Specific Sections (descriptions instead of locations/actions) ---
    ("Hello, I'd like a rotisserie chicken.", "A rotisserie chicken is a chicken that has been cooked by roasting. So you want a chicken that has undergone this cooking method."),
    ("Could you cut me 300g of Gruyère cheese?", "Cutting 300g of Gruyère cheese involves taking a piece of Gruyère and detaching a 300-gram portion. It's a cutting action."),
    ("Is this salmon farmed?", "Farmed salmon is salmon that grew up in a fish farm, as opposed to wild salmon. Your question concerns the origin of this salmon."),
    ("I'd like a baguette that's not too well-done.", "A baguette that's not too well-done is a baguette that has been lightly baked. You desire a baguette with this specific baking."),
    ("Can you recommend a wine to go with fish?", "Recommending a wine to accompany fish means suggesting a type of wine that pairs well with the taste of fish. This advice concerns food and wine pairing.")
])


# Let's check the size
print(f"Number of generated pairs: {len(negative_pairs_data)}")

Number of generated pairs: 240


In [5]:
# 1. Préparer les données pour le DataFrame
all_data = []

# Ajouter les paires positives avec un label de 1
for question, answer in positive_pairs_data:
    all_data.append({'question': question, 'answer': answer, 'label': 1})

# Ajouter les paires négatives avec un label de 0
for question, answer in negative_pairs_data:
    all_data.append({'question': question, 'answer': answer, 'label': 0})

# 2. Créer le DataFrame
df = pd.DataFrame(all_data)

# 3. (Optionnel mais recommandé) Mélanger le DataFrame
# Cela est utile si vous prévoyez d'utiliser ces données pour l'entraînement d'un modèle.
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
# random_state=42 assure que le mélange est reproductible

# Afficher les premières lignes du DataFrame et quelques informations
df

,question,answer,label
0,Is this fish fresh today?,"Yes, it arrived this morning.",1
1,Do you mind if I go ahead of you? I only have ...,"Yes, go ahead, no problem.",1
2,"Hello, I'll take this, please.","You'll take this? Okay, so you're taking this ...",0
3,Would it be possible to order a tiered cake fo...,"Yes, please speak to our pastry chef; he will ...",1
4,"Hello, I'd like two minced beef patties.","A red circle, a blue square, a green triangle.",0
...,...,...,...
482,"Excuse me, do you know where the pasta is?","Yes, pasta is in the grains and starches aisle...",1
483,Can I pay by contactless card?,Ducks go quack-quack.,0
484,Could you cut me 300g of Gruyère cheese?,Napoleon Bonaparte died on Saint Helena.,0
485,Can I pay by contactless card?,Pay by contactless card? It's a card payment t...,0


In [6]:
df.to_csv('../../data/data_train_en.csv', index=False)